# Milestone 3: Graph-RAG Implementation

**Theme:** Hotel

**Task:** Hybrid (Hotel Recommendation + Visa Assistant)

**Retrieval Approach:** Knowledge Graph + Embeddings (Graph-RAG)

---

## Table of Contents

1. [Part 1: Input Preprocessing](#part1)
   - [1.a Intent Classification](#part1a)
   - [1.b Entity Extraction](#part1b)
   - [1.c Input Embedding](#part1c)
2. [Part 2: Graph Retrieval + Experiments](#part2)
   - [2.a Baseline (Cypher Queries)](#part2a)
   - [2.b Embeddings-Based Retrieval (2 model comparison)](#part2b)
3. [Part 3: LLM Layer + Experiments](#part3)
   - [3.a Context Construction](#part3a)
   - [3.b Prompt Engineering](#part3b)
   - [3.c LLM Comparison (3 models)](#part3c)
4. [Part 4: UI + Full Pipeline Demo](#part4)
   - [4.a Streamlit Interface](#part4a)
   - [4.b End-to-End Demonstration](#part4b)

---
<a id='part1'></a>
# Part 1: Input Preprocessing

This section preprocesses natural language user queries into structured data for graph retrieval.

**Three components:**

**1.a Intent Classification** - Determines user's goal using action-based intents (LIST_HOTELS, RECOMMEND_HOTEL, DESCRIBE_HOTEL, COMPARE_HOTELS, CHECK_VISA)

**1.b Entity Extraction** - Extracts specific information (hotel names, cities, countries, traveler demographics, preferences)

**1.c Input Embedding** - Converts user query into vector representation for semantic similarity search (Part 2.b)

<a id='part1a'></a>
## 1.a Intent Classification

### Purpose
Classify user queries into specific intent categories to determine:
- What type of information the user needs
- Which Cypher queries to execute
- How to structure the response

### Design Decision: LLM-Based Classification

We use an **LLM-based approach** (OpenAI GPT-4o-mini) because:
- Handles natural language variations effectively
- Understands context and nuance
- Can apply complex tie-breaking rules
- Flexible for conversational queries

### Defined Intents

We define **5 action-based intents** that represent user goals:

| Intent | Purpose | Keywords | Example Queries |
|--------|---------|----------|----------------|
| **`LIST_HOTELS`** | Find multiple hotels matching filters | "show", "find", "list" | - "Show me hotels in Paris"<br>- "Find 5-star hotels in Dubai" |
| **`RECOMMEND_HOTEL`** | Get personalized suggestions or best options | "recommend", "suggest", "best", "top" | - "Recommend a hotel for families in Cairo"<br>- "What's the best cheap hotel in Rome?" |
| **`DESCRIBE_HOTEL`** | Get detailed info about a specific hotel | hotel name mentioned | - "Tell me about Hilton Cairo"<br>- "What is The Azure Tower like?" |
| **`COMPARE_HOTELS`** | Compare two or more hotels | "compare", "vs", "which is better" | - "Compare Hilton and Marriott in Cairo"<br>- "Which is better: Hotel A or Hotel B?" |
| **`CHECK_VISA`** | Visa requirements between countries | "visa", "entry requirement" | - "Do Egyptians need a visa for Turkey?"<br>- "Visa rules from UK to UAE" |

---

### Intent Design Philosophy

- **Action-based naming** — Each intent name describes what the user wants to DO
- **Clear semantic boundaries** — LIST (neutral) vs RECOMMEND (opinion) vs DESCRIBE (specific)
- **Explicit tie-breaking rules** — Clear keyword priorities reduce confusion

---

### Out-of-Scope Handling

If the query does not match any of the 5 intents  
(e.g., *"What's the weather?"*, *"Tell me a joke"*),  
the classifier returns `None`, and the system responds:

> **"I cannot help with this request. I can assist with hotel search, recommendations, visa information, and hotel comparisons."**

In [1]:
%pip install openai python-dotenv

Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 25.1.1 -> 25.3
[notice] To update, run: python.exe -m pip install --upgrade pip


In [2]:
from typing import Dict, List, Any, Optional
import os
from dotenv import load_dotenv
from openai import OpenAI

# Load environment variables
load_dotenv()

print("Libraries imported successfully")

Libraries imported successfully


### Implementation: LLM-Based Intent Classifier with OpenAI

The classifier uses a structured prompt with:
- Clear intent definitions
- Positive and negative examples for each intent
- Explicit tie-breaking rules to reduce ambiguity

In [3]:
class IntentClassifier:
    
    def __init__(self):
        # Load OpenAI API key
        self.api_key = os.getenv("OPENAI_API_KEY")
        if not self.api_key:
            raise ValueError("OPENAI_API_KEY not found in environment variables")
        
        # Initialize OpenAI client
        self.client = OpenAI(api_key=self.api_key)
        
        # Model configuration
        self.model = "gpt-4o-mini"  # Fast and cost-effective for classification
        
        # Define intents with descriptions
        self.intents = {
            "LIST_HOTELS": "Find multiple hotels matching filters (neutral listing)",
            "RECOMMEND_HOTEL": "Get personalized suggestions or best options (opinion/advice)",
            "DESCRIBE_HOTEL": "Get detailed information about one specific hotel",
            "COMPARE_HOTELS": "Compare two or more specific hotels",
            "CHECK_VISA": "Check visa requirements between countries"
        }
    
    def classify(self, user_query: str) -> Optional[str]:
        """
        Classify user query into one of 5 intents using LLM.
        Returns intent name or None if out-of-scope.
        """
        prompt = self._build_prompt(user_query)
        
        try:
            response = self.client.chat.completions.create(
                model=self.model,
                messages=[
                    {"role": "system", "content": "You are a precise intent classifier for a hotel search system. Return ONLY the intent name, nothing else."},
                    {"role": "user", "content": prompt}
                ],
                temperature=0.0,
                max_tokens=20
            )
            
            intent = response.choices[0].message.content.strip().upper()
            
            # Validate response
            if intent in self.intents.keys():
                return intent
            elif intent == "NONE":
                return None
            else:
                print(f"Warning: OpenAI returned unexpected value '{intent}', treating as out-of-scope")
                return None
                
        except Exception as e:
            print(f"Error calling OpenAI API: {e}")
            return None
    
    def _build_prompt(self, user_query: str) -> str:
        """Build the classification prompt with clear examples and tie-breaking rules."""
        
        return f"""Classify the following user query into ONE of these intents, or return "NONE" if it doesn't match any:

         ═══════════════════════════════════════════════════════════════════════════════

         **1. LIST_HOTELS**
            Purpose: User wants a neutral list of hotels matching filters
            Keywords: "show", "find", "list", "search for", "hotels in"
            
            Examples:
            - "Show me hotels in Paris"
            - "Find hotels in Dubai"
            - "List 5-star hotels in Rome"
            - "Hotels near the city center in Cairo"
            - "Search for hotels in Berlin"

         ═══════════════════════════════════════════════════════════════════════════════

         **2. RECOMMEND_HOTEL**
            Purpose: User wants advice, suggestions, or "best" options
            Keywords: "recommend", "suggest", "best", "top", "which should I", "what do you recommend"
            Can include personal preferences or demographics
            
            Examples:
            - "Recommend a hotel for families in Dubai"
            - "What's the best cheap hotel in Paris?"
            - "I'm 25 traveling solo, suggest a hotel in Cairo"
            - "Top hotels in Rome"
            - "Which hotel should I book in Berlin?"
            - "Suggest a romantic hotel for couples"

         ═══════════════════════════════════════════════════════════════════════════════

         **3. DESCRIBE_HOTEL**
            Purpose: Get detailed information about ONE specific hotel
            Keywords: "tell me about", "what is", "information on", "describe"
            Must mention a specific hotel name
            
            Examples:
            - "Tell me about Hilton Cairo"
            - "What is The Azure Tower like?"
            - "Information about Marriott Dubai"
            - "Describe the Grand Hotel Paris"
            - "What can you tell me about Hotel Adlon?"

         ═══════════════════════════════════════════════════════════════════════════════

         **4. COMPARE_HOTELS**
            Purpose: Compare TWO or MORE specific hotels
            Keywords: "compare", "vs", "versus", "which is better", "difference between"
            Must mention multiple hotels
            
            Examples:
            - "Compare Hilton Cairo and Marriott Cairo"
            - "Which is better: The Azure Tower or Nile Grandeur?"
            - "Hotel A vs Hotel B in Paris"
            - "Difference between Sheraton and Radisson in Dubai"
            - "Compare these three hotels: X, Y, and Z"

         ═══════════════════════════════════════════════════════════════════════════════

         **5. CHECK_VISA**
            Purpose: Check visa requirements between countries
            Keywords: "visa", "entry requirement", "travel document", "do I need"
            
            Examples:
            - "Do Egyptians need a visa for Turkey?"
            - "Visa requirements from UK to UAE"
            - "Do Indians need a visa to visit Germany?"
            - "What are the entry requirements for France?"
            - "Travel document requirements from US to Japan"

         ═══════════════════════════════════════════════════════════════════════════════

         **TIE-BREAKING RULES (VERY IMPORTANT):**

         1. If the query contains ANY of: "recommend", "suggest", "best", "top", "which should I"
            -> ALWAYS choose RECOMMEND_HOTEL (never LIST_HOTELS)

         2. If the query mentions personal info (age, traveler type: "family", "solo", "couple", "business")
            -> Choose RECOMMEND_HOTEL

         3. If the query mentions TWO or MORE hotel names
            -> Choose COMPARE_HOTELS (not DESCRIBE_HOTEL)

         4. If the query mentions ONLY ONE specific hotel name
            -> Choose DESCRIBE_HOTEL (not LIST_HOTELS)

         5. If the query is about "visa" or "entry requirements"
            -> Choose CHECK_VISA

         6. If the query uses only neutral search words ("show", "find", "list") WITHOUT opinion keywords
            -> Choose LIST_HOTELS

         7. If the query is COMPLETELY unrelated to hotels or travel
            -> Return NONE

         ═══════════════════════════════════════════════════════════════════════════════

         **User Query:** "{user_query}"

         **Your Response (return ONLY ONE of these):**
         LIST_HOTELS
         RECOMMEND_HOTEL
         DESCRIBE_HOTEL
         COMPARE_HOTELS
         CHECK_VISA
         NONE
         """

print("IntentClassifier class defined")

IntentClassifier class defined


### Testing Intent Classification

We test all 5 intents plus out-of-scope queries to validate the classifier's accuracy.

In [4]:
# Initialize classifier
classifier = IntentClassifier()

# Comprehensive test cases: (query, expected_intent)
test_cases = [
    # ===== LIST_HOTELS - neutral searching =====
    ("Show me hotels in Paris", "LIST_HOTELS"),
    ("Find 5-star hotels in Dubai", "LIST_HOTELS"),
    ("List hotels near the beach in Rome", "LIST_HOTELS"),
    ("Hotels in Cairo", "LIST_HOTELS"),
    ("Search for hotels in Berlin city center", "LIST_HOTELS"),
    
    # ===== RECOMMEND_HOTEL - advice/opinion/personalization =====
    ("Recommend a hotel for families in Dubai", "RECOMMEND_HOTEL"),
    ("What's the best cheap hotel in Paris?", "RECOMMEND_HOTEL"),
    ("I'm 25 traveling solo, suggest a clean hotel in Cairo", "RECOMMEND_HOTEL"),
    ("Top hotels in Rome", "RECOMMEND_HOTEL"),
    ("Which hotel should I book in Berlin?", "RECOMMEND_HOTEL"),
    ("Suggest a romantic hotel for our honeymoon", "RECOMMEND_HOTEL"),
    ("Best hotel for business travelers in London", "RECOMMEND_HOTEL"),
    
    # ===== DESCRIBE_HOTEL - single hotel info =====
    ("Tell me about Hilton Cairo", "DESCRIBE_HOTEL"),
    ("What is The Azure Tower like?", "DESCRIBE_HOTEL"),
    ("Information about Marriott Dubai", "DESCRIBE_HOTEL"),
    ("Describe the Grand Hotel Paris", "DESCRIBE_HOTEL"),
    
    # ===== COMPARE_HOTELS - compare multiple hotels =====
    ("Compare Hilton Cairo and Marriott Cairo", "COMPARE_HOTELS"),
    ("Which is better: The Azure Tower or Nile Grandeur?", "COMPARE_HOTELS"),
    ("Hotel A vs Hotel B in Paris", "COMPARE_HOTELS"),
    ("Difference between Sheraton and Radisson in Dubai", "COMPARE_HOTELS"),
    
    # ===== CHECK_VISA - visa requirements =====
    ("Do Egyptians need a visa for Turkey?", "CHECK_VISA"),
    ("Visa requirements from UK to UAE", "CHECK_VISA"),
    ("Do Indians need a visa to visit Germany?", "CHECK_VISA"),
    ("What are the entry requirements for France from USA?", "CHECK_VISA"),
    
    # ===== Out of scope - should return None =====
    ("What's the weather like today?", None),
    ("Tell me a joke", None),
    ("How do I cook pasta?", None),
    ("What's the capital of France?", None),
]

print("=" * 100)
print("TESTING INTENT CLASSIFICATION")
print("=" * 100)

num_total = len(test_cases)
num_correct = 0
failures = []

for i, (query, expected) in enumerate(test_cases, 1):
    predicted = classifier.classify(query)
    is_correct = (predicted == expected)
    
    if is_correct:
        num_correct += 1
        status = "[PASS]"
    else:
        status = "[FAIL]"
        failures.append((query, expected, predicted))
    
    # Format output
    print(f"\n[{i}/{num_total}] {status}")
    print(f"Query:     {query}")
    print(f"Expected:  {expected}")
    print(f"Predicted: {predicted}")
    
    if predicted is None:
        print(f"System Response: 'I cannot help with this request. I can assist with hotel search, recommendations, visa information, and hotel comparisons.'")

print("\n" + "=" * 100)
print(f"SUMMARY: {num_correct}/{num_total} correct ({(num_correct / num_total) * 100:.1f}% accuracy)")
print("=" * 100)

if failures:
    print(f"\nFAILED CASES ({len(failures)}):")
    for query, expected, predicted in failures:
        print(f"  - '{query}'")
        print(f"    Expected: {expected}, Got: {predicted}")
else:
    print("\nALL TESTS PASSED!")

TESTING INTENT CLASSIFICATION

[1/28] [PASS]
Query:     Show me hotels in Paris
Expected:  LIST_HOTELS
Predicted: LIST_HOTELS

[2/28] [PASS]
Query:     Find 5-star hotels in Dubai
Expected:  LIST_HOTELS
Predicted: LIST_HOTELS

[3/28] [PASS]
Query:     List hotels near the beach in Rome
Expected:  LIST_HOTELS
Predicted: LIST_HOTELS

[4/28] [PASS]
Query:     Hotels in Cairo
Expected:  LIST_HOTELS
Predicted: LIST_HOTELS

[5/28] [PASS]
Query:     Search for hotels in Berlin city center
Expected:  LIST_HOTELS
Predicted: LIST_HOTELS

[6/28] [PASS]
Query:     Recommend a hotel for families in Dubai
Expected:  RECOMMEND_HOTEL
Predicted: RECOMMEND_HOTEL

[7/28] [PASS]
Query:     What's the best cheap hotel in Paris?
Expected:  RECOMMEND_HOTEL
Predicted: RECOMMEND_HOTEL

[8/28] [PASS]
Query:     I'm 25 traveling solo, suggest a clean hotel in Cairo
Expected:  RECOMMEND_HOTEL
Predicted: RECOMMEND_HOTEL

[9/28] [PASS]
Query:     Top hotels in Rome
Expected:  RECOMMEND_HOTEL
Predicted: RECOMMEND_HO

---
<a id='part1b'></a>
## 1.b Entity Extraction

### Purpose
Extract structured information from user queries to fill Cypher query parameters.

### Design Decision: LLM + Normalization + Schema Enforcement

We use a **three-stage approach**:
1. **LLM Extraction** (OpenAI GPT-4o-mini) - Extract raw entities as JSON
2. **Schema Enforcement** - Validate against intent-specific schemas
3. **Normalization** - Standardize values (star_rating → int, aspects → list, etc.)

**Critical Behavioral Rules:**
- Extract ONLY explicitly mentioned entities
- Vague words like "good", "best", "nice" do **NOT** trigger aspect extraction
- Aspects extracted **ONLY** if specifically mentioned (e.g., "clean rooms" → cleanliness)

In [5]:
import json

# Initialize OpenAI client for entity extraction (reusing same model as intent classifier)
client = OpenAI(api_key=os.getenv("OPENAI_API_KEY"))

print("OpenAI client initialized for entity extraction")

OpenAI client initialized for entity extraction


### Entity Schemas by Intent

Each intent requires different entities:

| Intent | Entities |
|--------|----------|
| **LIST_HOTELS** | city, country, star_rating |
| **RECOMMEND_HOTEL** | city, country, traveller_type, age_group, user_gender, star_rating, aspects |
| **DESCRIBE_HOTEL** | hotel_name, aspects |
| **COMPARE_HOTELS** | hotel1, hotel2, traveller_type, aspects |
| **CHECK_VISA** | from_country, to_country |

In [6]:
# ============================================================================
# Intent-Aware Schemas
# ============================================================================

SCHEMAS: Dict[str, Dict[str, Any]] = {
    "LIST_HOTELS": {
        "city": None,
        "country": None,
        "star_rating": None
    },
    "RECOMMEND_HOTEL": {
        "city": None,
        "country": None,
        "traveller_type": None,
        "age_group": None,
        "user_gender": None,
        "star_rating": None,
        "aspects": None
    },
    "DESCRIBE_HOTEL": {
        "hotel_name": None,
        "aspects": None
    },
    "COMPARE_HOTELS": {
        "hotel1": None,
        "hotel2": None,
        "traveller_type": None,
        "aspects": None
    },
    "CHECK_VISA": {
        "from_country": None,
        "to_country": None
    }
}

# Allowed aspects from dataset
ALLOWED_ASPECTS = [
    "cleanliness", "comfort", "facilities",
    "location", "staff", "value_for_money"
]

print("Schemas defined successfully")
print(f"Intents: {list(SCHEMAS.keys())}")
print(f"Allowed aspects: {ALLOWED_ASPECTS}")

Schemas defined successfully
Intents: ['LIST_HOTELS', 'RECOMMEND_HOTEL', 'DESCRIBE_HOTEL', 'COMPARE_HOTELS', 'CHECK_VISA']
Allowed aspects: ['cleanliness', 'comfort', 'facilities', 'location', 'staff', 'value_for_money']


In [7]:
def enforce_schema(intent: str, entities: Dict[str, Any]) -> Dict[str, Any]:
    """
    Enforce schema constraints and normalize values.
    
    Rules:
    - Drop keys not in SCHEMAS[intent]
    - Ensure all schema keys exist (missing → None)
    - Normalize star_rating → int (1-5)
    - Normalize aspects → list[str], lowercase, deduplicated
    - Normalize traveller_type, user_gender to valid values
    """
    schema = SCHEMAS[intent]
    result = {}
    
    for key in schema.keys():
        value = entities.get(key, None)
        
        # Normalize star_rating → int
        if key == "star_rating" and value is not None:
            try:
                value = int(value)
                if not (1 <= value <= 5):
                    value = None
            except (ValueError, TypeError):
                value = None
        
        # Normalize aspects → list[str]
        elif key == "aspects" and value is not None:
            if isinstance(value, str):
                value = [value]
            if isinstance(value, list):
                # Lowercase, snake_case, deduplicate, filter allowed
                normalized = []
                for asp in value:
                    if isinstance(asp, str):
                        asp_clean = asp.lower().strip().replace(" ", "_").replace("-", "_")
                        if asp_clean in ALLOWED_ASPECTS:
                            normalized.append(asp_clean)
                value = list(set(normalized)) if normalized else None
            else:
                value = None
        
        # Normalize traveller_type
        elif key == "traveller_type" and value is not None:
            if isinstance(value, str):
                value = value.lower().strip()
                if value not in ["family", "solo", "couple", "business", "group"]:
                    value = None
        
        # Normalize user_gender
        elif key == "user_gender" and value is not None:
            if isinstance(value, str):
                value = value.lower().strip()
                if value not in ["male", "female"]:
                    value = None
        
        result[key] = value
    
    return result

print("enforce_schema() function defined")

enforce_schema() function defined


In [8]:
def extract_entities(text: str, intent: str) -> Dict[str, Any]:
    """
    Extract entities from user query using GPT-4o-mini (same model as intent classifier).
    
    Args:
        text: User query
        intent: One of the 5 intents (LIST_HOTELS, RECOMMEND_HOTEL, etc.)
    
    Returns:
        Schema-compliant dictionary with extracted entities
    """
    if intent not in SCHEMAS:
        raise ValueError(f"Unknown intent: {intent}")
    
    # Build extraction prompt
    prompt = f"""Extract entities from this user query and return ONLY valid JSON.

Query: "{text}"
Intent: {intent}
Required keys: {list(SCHEMAS[intent].keys())}

CRITICAL RULES:
1. Extract ONLY explicitly mentioned entities
2. Missing information → null
3. Do NOT interpret "good", "best", "nice", "top" as entities
4. For aspects: ONLY extract if explicitly mentioned
   - "clean" → cleanliness
   - "comfortable" → comfort
   - "good location" → location
   - "staff" → staff
   - "facilities" → facilities
   - "cheap"/"value" → value_for_money
   
5. Do NOT extract aspects from vague words:
   - "good", "best", "nice", "top", "quality" → extract NO aspects

6. Allowed aspects: {ALLOWED_ASPECTS}
7. traveller_type: family, solo, couple, business, group
8. user_gender: male, female
9. star_rating: 1-5 (numeric only)

Return ONLY JSON matching this schema: {SCHEMAS[intent]}"""

    try:
        response = client.chat.completions.create(
            model="gpt-4o-mini",  # Same model as intent classifier
            messages=[
                {"role": "system", "content": "You are a precise NER system. Return ONLY valid JSON, no markdown, no explanations."},
                {"role": "user", "content": prompt}
            ],
            temperature=0.0,
            max_tokens=200
        )
        
        raw = response.choices[0].message.content.strip()
        
        # Strip markdown code blocks if present
        if raw.startswith("```"):
            raw = raw.split("```")[1]
            if raw.startswith("json"):
                raw = raw[4:]
            raw = raw.strip()
        
        entities = json.loads(raw)
        
        # Enforce schema constraints
        return enforce_schema(intent, entities)
        
    except Exception as e:
        print(f"Error extracting entities: {e}")
        # Return empty schema on error
        return dict(SCHEMAS[intent])

print("extract_entities() function defined")

extract_entities() function defined


### Testing Entity Extraction

We test the NER system with three critical examples:
1. **Explicit aspects** - Aspects clearly mentioned should be extracted
2. **Vague quality words** - "good", "best" should NOT trigger aspect extraction
3. **Comparison query** - Extract multiple hotels and specific aspects

In [9]:
print("=" * 100)
print("TESTING ENTITY EXTRACTION")
print("=" * 100)

# ============================================================================
# Example 1: Explicit aspects mentioned
# ============================================================================
query1 = "Recommend a hotel for families in Dubai with good location and clean rooms"
intent1 = "RECOMMEND_HOTEL"
result1 = extract_entities(query1, intent1)

print("\n[Example 1] Explicit Aspects")
print(f"Query:  {query1}")
print(f"Intent: {intent1}")
print(f"Output: {json.dumps(result1, indent=2)}")
print("Expected: traveller_type='family', city='Dubai', aspects=['location', 'cleanliness']")
print("Note: 'good location' → location aspect, 'clean rooms' → cleanliness aspect")

# ============================================================================
# Example 2: Vague quality words (NO aspects should be extracted)
# ============================================================================
query2 = "I want good hotels for family in Paris"
intent2 = "RECOMMEND_HOTEL"
result2 = extract_entities(query2, intent2)

print("\n[Example 2] Vague Quality Words (Critical Test)")
print(f"Query:  {query2}")
print(f"Intent: {intent2}")
print(f"Output: {json.dumps(result2, indent=2)}")
print("Expected: traveller_type='family', city='Paris', aspects=None")
print("Note: 'good' is vague and should NOT extract any aspect")

# ============================================================================
# Example 3: Compare hotels with explicit aspects
# ============================================================================
query3 = "Compare Hilton Cairo and Marriott Cairo for staff quality and comfort"
intent3 = "COMPARE_HOTELS"
result3 = extract_entities(query3, intent3)

print("\n[Example 3] Comparison with Explicit Aspects")
print(f"Query:  {query3}")
print(f"Intent: {intent3}")
print(f"Output: {json.dumps(result3, indent=2)}")
print("Expected: hotel1='Hilton Cairo', hotel2='Marriott Cairo', aspects=['staff', 'comfort']")
print("Note: 'staff quality' → staff aspect, 'comfort' → comfort aspect")

print("\n" + "=" * 100)
print("ENTITY EXTRACTION TESTING COMPLETE")
print("=" * 100)
print("=" * 100)
print("EXTENDED ENTITY EXTRACTION STRESS TEST")
print("=" * 100)

# =====================================================================
# Example 4: Multiple aspects + traveller type + star rating
# =====================================================================
query4 = "Recommend a 5 star hotel for business travelers in London with good staff and comfort"
intent4 = "RECOMMEND_HOTEL"
result4 = extract_entities(query4, intent4)

print("\n[Example 4] Multiple Explicit Aspects + Star Rating")
print(f"Query:  {query4}")
print(f"Intent: {intent4}")
print(f"Output: {json.dumps(result4, indent=2)}")
print("Expected: city='London', star_rating=5, traveller_type='business', aspects=['staff','comfort']")
print("Note: 'good staff' → staff, 'comfort' → comfort")

# =====================================================================
# Example 5: No aspects, no city (pure traveller preference)
# =====================================================================
query5 = "I want hotels for solo travelers"
intent5 = "RECOMMEND_HOTEL"
result5 = extract_entities(query5, intent5)

print("\n[Example 5] Traveller Type Only")
print(f"Query:  {query5}")
print(f"Intent: {intent5}")
print(f"Output: {json.dumps(result5, indent=2)}")
print("Expected: traveller_type='solo', all other fields=None")

# =====================================================================
# Example 6: LIST_HOTELS should ignore traveller types and aspects
# =====================================================================
query6 = "List hotels in Rome for families with clean rooms"
intent6 = "LIST_HOTELS"
result6 = extract_entities(query6, intent6)

print("\n[Example 6] LIST_HOTELS Ignores Aspects & Traveller Type")
print(f"Query:  {query6}")
print(f"Intent: {intent6}")
print(f"Output: {json.dumps(result6, indent=2)}")
print("Expected: city='Rome', aspects ignored, traveller_type ignored")

# =====================================================================
# Example 7: DESCRIBE_HOTEL without aspects
# =====================================================================
query7 = "Describe Hilton Dubai"
intent7 = "DESCRIBE_HOTEL"
result7 = extract_entities(query7, intent7)

print("\n[Example 7] Describe Hotel (No Aspects)")
print(f"Query:  {query7}")
print(f"Intent: {intent7}")
print(f"Output: {json.dumps(result7, indent=2)}")
print("Expected: hotel_name='Hilton Dubai', aspects=None")

# =====================================================================
# Example 8: DESCRIBE_HOTEL with aspects
# =====================================================================
query8 = "Describe Hilton Dubai focusing on location and facilities"
intent8 = "DESCRIBE_HOTEL"
result8 = extract_entities(query8, intent8)

print("\n[Example 8] Describe Hotel with Explicit Aspects")
print(f"Query:  {query8}")
print(f"Intent: {intent8}")
print(f"Output: {json.dumps(result8, indent=2)}")
print("Expected: hotel_name='Hilton Dubai', aspects=['location','facilities']")

# =====================================================================
# Example 9: CHECK_VISA intent
# =====================================================================
query9 = "Do Egyptians need a visa for Turkey?"
intent9 = "CHECK_VISA"
result9 = extract_entities(query9, intent9)

print("\n[Example 9] Visa Query")
print(f"Query:  {query9}")
print(f"Intent: {intent9}")
print(f"Output: {json.dumps(result9, indent=2)}")
print("Expected: from_country='Egypt', to_country='Turkey'")

# =====================================================================
# Example 10: Noise words + vague adjectives
# =====================================================================
query10 = "I want really amazing and top hotels for families in Madrid"
intent10 = "RECOMMEND_HOTEL"
result10 = extract_entities(query10, intent10)

print("\n[Example 10] Noise & Vague Adjectives")
print(f"Query:  {query10}")
print(f"Intent: {intent10}")
print(f"Output: {json.dumps(result10, indent=2)}")
print("Expected: city='Madrid', traveller_type='family', aspects=None")
print("Note: 'amazing', 'top' must be ignored")

# =====================================================================
# Example 11: Multiple cities (edge case)
# =====================================================================
query11 = "Compare hotels in Paris and London for location"
intent11 = "COMPARE_HOTELS"
result11 = extract_entities(query11, intent11)


print("\n" + "=" * 100)
print("EXTENDED ENTITY EXTRACTION TESTING COMPLETE")
print("=" * 100)


TESTING ENTITY EXTRACTION

[Example 1] Explicit Aspects
Query:  Recommend a hotel for families in Dubai with good location and clean rooms
Intent: RECOMMEND_HOTEL
Output: {
  "city": "Dubai",
  "country": null,
  "traveller_type": "family",
  "age_group": null,
  "user_gender": null,
  "star_rating": null,
  "aspects": [
    "location",
    "cleanliness"
  ]
}
Expected: traveller_type='family', city='Dubai', aspects=['location', 'cleanliness']
Note: 'good location' → location aspect, 'clean rooms' → cleanliness aspect

[Example 2] Vague Quality Words (Critical Test)
Query:  I want good hotels for family in Paris
Intent: RECOMMEND_HOTEL
Output: {
  "city": "Paris",
  "country": null,
  "traveller_type": "family",
  "age_group": null,
  "user_gender": null,
  "star_rating": null,
  "aspects": null
}
Expected: traveller_type='family', city='Paris', aspects=None
Note: 'good' is vague and should NOT extract any aspect

[Example 3] Comparison with Explicit Aspects
Query:  Compare Hilton Cair

---
<a id='part2'></a>
# Part 2: Graph Retrieval + Experiments

This section implements graph-based retrieval using Cypher queries to extract relevant information from the Neo4j knowledge graph.

**Two approaches:**
- **2.a Baseline (Cypher Queries)** - Structured queries based on extracted entities
- **2.b Embeddings-Based Retrieval** - Semantic similarity search using vector embeddings

---
<a id='part2a'></a>
## 2.a Baseline (Cypher Queries)

### Purpose
Execute structured Cypher queries against the Neo4j knowledge graph to retrieve hotels, reviews, and visa information based on extracted entities from Part 1.

### Design Decision: Intent-Aware Query Functions

We create one query function per intent that handles all variations internally:
- Each function builds dynamic WHERE clauses based on provided entities
- Handles null values by conditionally adding filters
- Returns consistent list of dictionaries format
- Empty list `[]` when no results found


### Step 1: Neo4j Connection Setup

We create a connection manager class to handle Neo4j database connections. This reads configuration from the KnowledgeGraph directory and provides a session for executing queries.


In [10]:
%pip install neo4j


Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 25.1.1 -> 25.3
[notice] To update, run: python.exe -m pip install --upgrade pip


In [11]:
from neo4j import GraphDatabase
import os

class Neo4jConnection:
    """
    Manages Neo4j database connection and provides query execution interface.
    
    Reads configuration from KnowledgeGraph/config.txt to connect to the database.
    """
    
    def __init__(self, config_path=None):
        """
        Initialize connection to Neo4j database.
        
        Args:
            config_path: Path to config.txt file. If None, tries multiple default paths.
        """
        if config_path is None:
            # Try multiple paths to find config.txt
            possible_paths = [
                os.path.join('KnowledgeGraph', 'config.txt'),
                os.path.join('Milestone 3', 'KnowledgeGraph', 'config.txt'),
                os.path.join(os.getcwd(), 'KnowledgeGraph', 'config.txt')
            ]
            config_path = None
            for path in possible_paths:
                if os.path.exists(path):
                    config_path = path
                    break
            if config_path is None:
                raise FileNotFoundError(f"Could not find config.txt. Tried: {possible_paths}")
        
        config = self._read_config(config_path)
        self.uri = config['URI']
        self.user = config['USERNAME']
        self.password = config['PASSWORD']
        self.driver = GraphDatabase.driver(self.uri, auth=(self.user, self.password))
    
    def _read_config(self, config_file):
        """Read Neo4j configuration from config file."""
        config = {}
        with open(config_file, 'r') as f:
            for line in f:
                line = line.strip()
                if line and '=' in line:
                    key, value = line.split('=', 1)
                    config[key] = value
        return config
    
    def execute_query(self, query, parameters=None):
        """
        Execute a Cypher query and return results.
        
        Args:
            query: Cypher query string
            parameters: Dictionary of query parameters (can be None or empty dict)
        
        Returns:
            List of result records as dictionaries
        """
        if not query:
            return []
        
        with self.driver.session() as session:
            result = session.run(query, parameters or {})
            return [dict(record) for record in result]
    
    def close(self):
        """Close the database connection."""
        self.driver.close()

print("Neo4jConnection class defined")


Neo4jConnection class defined


### Step 2: Query Functions

We create one query function per intent. Each function:
1. Takes extracted entities as input
2. Builds a parameterized Cypher query with appropriate filters
3. Returns structured results that can be used by the LLM layer

**Query Design Principles:**
- Use parameterized queries to prevent injection attacks
- Handle optional filters (None values mean no filter)
- Return consistent data structures for each intent
- Include relevant hotel properties and aggregated review scores


#### Query Library: 10+ Cypher Query Templates

Purpose: Create a library of parameterized Cypher query templates for different question types and entity combinations.

**Design Pattern:** Template Selection
- Each template handles a specific combination of entities and filters
- A query selector function chooses the appropriate template based on extracted entities
- Templates are organized by intent with clear naming conventions

**Query Coverage (14 templates total):**

**LIST_HOTELS (5 templates):**
1. Template L1: List by city only
2. Template L2: List by country only
3. Template L3: List by star rating only
4. Template L4: List by city + star rating
5. Template L5: List by country + star rating

**RECOMMEND_HOTEL (4 templates):**
6. Template R1: Recommend by location only (no demographics/aspects)
7. Template R3: Recommend by aspects only (no demographics)
8. Template R4: Recommend by traveller type + location + aspects (with fallback)
9. Template R5: Recommend with star rating filter

**DESCRIBE_HOTEL (2 templates):**
10. Template D1: Describe hotel (all aspects)
11. Template D2: Describe hotel (specific aspects)

**COMPARE_HOTELS (2 templates):**
12. Template C1: Compare hotels (all aspects)
13. Template C2: Compare hotels (with traveller type demographic)

**CHECK_VISA (1 template):**
14. Template V1: Check visa requirement

In [12]:

# ============================================================================
# QUERY LIBRARY: 14 Cypher Query Templates for Different Question Types
# ============================================================================

class QueryLibrary:
    """
    Comprehensive library of parameterized Cypher query templates.
    
    Organized by intent with templates for different entity combinations.
    Each template is designed to answer specific question types.
    """
    
    # ========================================================================
    # LIST_HOTELS TEMPLATES (5 templates)
    # ========================================================================
    
    @staticmethod
    def template_L1_list_by_city(conn: Neo4jConnection, city: str) -> List[Dict[str, Any]]:
        """Template L1: List hotels in a specific city (most common query)"""
        query = """
        MATCH (h:Hotel)-[:LOCATED_IN]->(city:City)-[:LOCATED_IN]->(country:Country)
        WHERE city.name = $city
        RETURN h.name AS name,
               city.name AS city,
               country.name AS country,
               h.star_rating AS star_rating
        ORDER BY h.star_rating DESC, h.name
        LIMIT 50
        """
        return conn.execute_query(query, {'city': city})
    
    @staticmethod
    def template_L2_list_by_country(conn: Neo4jConnection, country: str) -> List[Dict[str, Any]]:
        """Template L2: List all hotels in a country"""
        query = """
        MATCH (h:Hotel)-[:LOCATED_IN]->(city:City)-[:LOCATED_IN]->(country:Country)
        WHERE country.name = $country
        RETURN h.name AS name,
               city.name AS city,
               country.name AS country,
               h.star_rating AS star_rating
        ORDER BY h.star_rating DESC, h.name
        LIMIT 50
        """
        return conn.execute_query(query, {'country': country})
    
    @staticmethod
    def template_L3_list_by_rating(conn: Neo4jConnection, star_rating: int) -> List[Dict[str, Any]]:
        """Template L3: List hotels with specific star rating"""
        query = """
        MATCH (h:Hotel)-[:LOCATED_IN]->(city:City)-[:LOCATED_IN]->(country:Country)
        WHERE h.star_rating = $star_rating
        RETURN h.name AS name,
               city.name AS city,
               country.name AS country,
               h.star_rating AS star_rating
        ORDER BY h.name
        LIMIT 50
        """
        return conn.execute_query(query, {'star_rating': star_rating})
    
    @staticmethod
    def template_L4_list_by_city_and_rating(conn: Neo4jConnection, city: str, star_rating: int) -> List[Dict[str, Any]]:
        """Template L4: List hotels in city with specific star rating"""
        query = """
        MATCH (h:Hotel)-[:LOCATED_IN]->(city:City)-[:LOCATED_IN]->(country:Country)
        WHERE city.name = $city AND h.star_rating = $star_rating
        RETURN h.name AS name,
               city.name AS city,
               country.name AS country,
               h.star_rating AS star_rating
        ORDER BY h.name
        LIMIT 50
        """
        return conn.execute_query(query, {'city': city, 'star_rating': star_rating})
    
    @staticmethod
    def template_L5_list_by_country_and_rating(conn: Neo4jConnection, country: str, star_rating: int) -> List[Dict[str, Any]]:
        """Template L5: List hotels in country with specific star rating"""
        query = """
        MATCH (h:Hotel)-[:LOCATED_IN]->(city:City)-[:LOCATED_IN]->(country:Country)
        WHERE country.name = $country AND h.star_rating = $star_rating
        RETURN h.name AS name,
               city.name AS city,
               country.name AS country,
               h.star_rating AS star_rating
        ORDER BY h.name
        LIMIT 50
        """
        return conn.execute_query(query, {'country': country, 'star_rating': star_rating})
    
    # ========================================================================
    # RECOMMEND_HOTEL TEMPLATES (5 templates)
    # ========================================================================
    
    @staticmethod
    def template_R1_recommend_by_location(conn: Neo4jConnection, city: str) -> List[Dict[str, Any]]:
        """Template R1: Recommend best hotels in location (ranked by overall score)"""
        query = """
        MATCH (h:Hotel)-[:LOCATED_IN]->(city:City)-[:LOCATED_IN]->(country:Country)
        WHERE city.name = $city
        OPTIONAL MATCH (h)<-[:REVIEWED]-(r:Review)
        WITH h, city, country, collect(r) AS reviews
        WHERE size(reviews) > 0
        UNWIND reviews AS r
        RETURN h.name AS name,
               city.name AS city,
               country.name AS country,
               avg(r.score_overall) AS overall_score
        ORDER BY overall_score DESC
        LIMIT 10
        """
        return conn.execute_query(query, {'city': city})
    
    @staticmethod
    def template_R3_recommend_by_aspects(conn: Neo4jConnection, city: str, aspects: List[str], age_group: str = None, user_gender: str = None) -> List[Dict[str, Any]]:
        """Template R3: Recommend hotels based on specific aspects (with optional demographics)"""
        aspect_mapping = {
            'cleanliness': 'score_cleanliness',
            'comfort': 'score_comfort',
            'facilities': 'score_facilities',
            'location': 'score_location',
            'staff': 'score_staff',
            'value_for_money': 'score_value_for_money'
        }
        
        # Validate and filter aspects
        valid_aspects = [a for a in (aspects or []) if a in aspect_mapping]
        if not valid_aspects:
            return []

        # Build per-aspect review fields and composite score
        aspect_avg = " + ".join([f"coalesce(avg(r.{aspect_mapping[a]}), 0)" for a in valid_aspects])
        aspect_count = len(valid_aspects)
        aspect_select = ", ".join([f"avg(r.{aspect_mapping[a]}) AS {a}_review" for a in valid_aspects])

        # Build WHERE clause for demographic filters
        demo_conditions = []
        params = {'city': city}
        
        if age_group:
            demo_conditions.append("u.age_group = $age_group")
            params['age_group'] = age_group
        
        if user_gender:
            demo_conditions.append("u.gender = $user_gender")
            params['user_gender'] = user_gender
        
        # Build complete query with optional demographic filtering
        demo_clause = ""
        user_match = ""
        if demo_conditions:
            user_match = "<-[:WROTE]-(u:User)"
            demo_clause = " AND " + " AND ".join(demo_conditions)

        query = f"""
        MATCH (h:Hotel)-[:LOCATED_IN]->(city:City)-[:LOCATED_IN]->(country:Country)
        WHERE city.name = $city
        OPTIONAL MATCH (h)<-[:REVIEWED]-(r:Review){user_match}
        WHERE TRUE{demo_clause}
        WITH h, city, country, collect(r) AS reviews
        WHERE size(reviews) > 0
        UNWIND reviews AS r
        RETURN h.name AS name,
               city.name AS city,
               country.name AS country,
               {aspect_select},
               ({aspect_avg}) / {aspect_count} AS composite_aspect_score,
               count(r) AS review_count
        ORDER BY composite_aspect_score DESC
        LIMIT 10
        """
        return conn.execute_query(query, params)
    
    @staticmethod
    def template_R4_recommend_by_traveller_and_aspects(conn: Neo4jConnection, city: str, traveller_type: str, aspects: List[str], age_group: str = None, user_gender: str = None) -> List[Dict[str, Any]]:
        """Template R4: Recommend for traveller type + specific aspects (with optional demographics)"""
        aspect_mapping = {
            'cleanliness': 'score_cleanliness',
            'comfort': 'score_comfort',
            'facilities': 'score_facilities',
            'location': 'score_location',
            'staff': 'score_staff',
            'value_for_money': 'score_value_for_money'
        }
        
        # Validate and filter aspects
        valid_aspects = [a for a in (aspects or []) if a in aspect_mapping]
        if not valid_aspects:
            # Fallback to a generic recommend-by-aspects if no valid aspects
            return QueryLibrary.template_R3_recommend_by_aspects(conn, city, aspects, age_group, user_gender)

        aspect_avg = " + ".join([f"coalesce(avg(r.{aspect_mapping[a]}), 0)" for a in valid_aspects])
        aspect_count = len(valid_aspects)
        aspect_select = ", ".join([f"avg(r.{aspect_mapping[a]}) AS {a}_review" for a in valid_aspects])

        # Build WHERE clause for demographic and traveller filters
        conditions = ["t.type = $traveller_type"]
        params = {'city': city, 'traveller_type': traveller_type}
        
        if age_group:
            conditions.append("u.age_group = $age_group")
            params['age_group'] = age_group
        
        if user_gender:
            conditions.append("u.gender = $user_gender")
            params['user_gender'] = user_gender
        
        # Build complete query with optional demographic filtering
        where_clause = " AND ".join(conditions)
        user_match = "<-[:WROTE]-(u:User)" if (age_group or user_gender) else ""

        query = f"""
        MATCH (h:Hotel)-[:LOCATED_IN]->(city:City)-[:LOCATED_IN]->(country:Country)
        WHERE city.name = $city
        OPTIONAL MATCH (h)<-[:REVIEWED]-(r:Review)<-[:WROTE]-(t:Traveller){user_match}
        WHERE {where_clause}
        WITH h, city, country, collect(r) AS reviews
        WHERE size(reviews) > 0
        UNWIND reviews AS r
        RETURN h.name AS name,
               city.name AS city,
               country.name AS country,
               {aspect_select},
               ({aspect_avg}) / {aspect_count} AS composite_aspect_score,
               count(r) AS review_count
        ORDER BY composite_aspect_score DESC
        LIMIT 10
        """

        results = conn.execute_query(query, params)

        # Fallback if no results with demographics/traveller type
        if not results:
            results = QueryLibrary.template_R3_recommend_by_aspects(conn, city, aspects, age_group, user_gender)

        return results
    
    @staticmethod
    def template_R5_recommend_with_rating_filter(conn: Neo4jConnection, city: str, star_rating: int) -> List[Dict[str, Any]]:
        """Template R5: Recommend in city filtered by minimum star rating"""
        query = """
        MATCH (h:Hotel)-[:LOCATED_IN]->(city:City)-[:LOCATED_IN]->(country:Country)
        WHERE city.name = $city AND h.star_rating >= $star_rating
        OPTIONAL MATCH (h)<-[:REVIEWED]-(r:Review)
        WITH h, city, country, collect(r) AS reviews
        WHERE size(reviews) > 0
        UNWIND reviews AS r
        RETURN h.name AS name,
               city.name AS city,
               country.name AS country,
               avg(r.score_overall) AS overall_score
        ORDER BY overall_score DESC
        LIMIT 10
        """
        return conn.execute_query(query, {'city': city, 'star_rating': star_rating})
    
    # ========================================================================
    # DESCRIBE_HOTEL TEMPLATES (2 templates)
    # ========================================================================
    
    @staticmethod
    def template_D1_describe_all_aspects(conn: Neo4jConnection, hotel_name: str) -> List[Dict[str, Any]]:
        """Template D1: Describe hotel with all 6 aspects"""
        query = """
        MATCH (h:Hotel)-[:LOCATED_IN]->(city:City)-[:LOCATED_IN]->(country:Country)
        WHERE toLower(h.name) = toLower($hotel_name)
        OPTIONAL MATCH (h)<-[:REVIEWED]-(r:Review)
        RETURN h.name AS name,
               city.name AS city,
               country.name AS country,
               h.cleanliness_base AS cleanliness_base,
               h.comfort_base AS comfort_base,
               h.facilities_base AS facilities_base,
               h.location_base AS location_base,
               h.staff_base AS staff_base,
               h.value_for_money_base AS value_for_money_base,
               avg(r.score_cleanliness) AS cleanliness_review,
               avg(r.score_comfort) AS comfort_review,
               avg(r.score_facilities) AS facilities_review,
               avg(r.score_location) AS location_review,
               avg(r.score_staff) AS staff_review,
               avg(r.score_value_for_money) AS value_for_money_review,
               count(r) AS review_count
        LIMIT 1
        """
        return conn.execute_query(query, {'hotel_name': hotel_name})
    
    @staticmethod
    def template_D2_describe_specific_aspects(conn: Neo4jConnection, hotel_name: str, aspects: List[str]) -> List[Dict[str, Any]]:
        """Template D2: Describe hotel focusing on specific aspects"""
        aspect_mapping = {
            'cleanliness': ('cleanliness_base', 'score_cleanliness'),
            'comfort': ('comfort_base', 'score_comfort'),
            'facilities': ('facilities_base', 'score_facilities'),
            'location': ('location_base', 'score_location'),
            'staff': ('staff_base', 'score_staff'),
            'value_for_money': ('value_for_money_base', 'score_value_for_money')
        }
        
        valid_aspects = [a for a in aspects if a in aspect_mapping]
        if not valid_aspects:
            # Fallback to all aspects
            return QueryLibrary.template_D1_describe_all_aspects(conn, hotel_name)
        
        aspect_fields = []
        for aspect in valid_aspects:
            base_field, review_field = aspect_mapping[aspect]
            aspect_fields.append(f"h.{base_field} AS {aspect}_base")
            aspect_fields.append(f"avg(r.{review_field}) AS {aspect}_review")
        
        aspect_select = ", ".join(aspect_fields)
        
        query = f"""
        MATCH (h:Hotel)-[:LOCATED_IN]->(city:City)-[:LOCATED_IN]->(country:Country)
        WHERE toLower(h.name) = toLower($hotel_name)
        OPTIONAL MATCH (h)<-[:REVIEWED]-(r:Review)
        RETURN h.name AS name,
               city.name AS city,
               country.name AS country,
               {aspect_select},
               count(r) AS review_count
        LIMIT 1
        """
        return conn.execute_query(query, {'hotel_name': hotel_name})
    
    # ========================================================================
    # COMPARE_HOTELS TEMPLATES (2 templates)
    # ========================================================================
    
    @staticmethod
    def template_C1_compare_all_aspects(conn: Neo4jConnection, hotel1: str, hotel2: str, aspects: List[str] = None) -> List[Dict[str, Any]]:
        """Template C1: Compare two hotels across specified or all aspects"""
        aspect_mapping = {
            'cleanliness': ('cleanliness_base', 'score_cleanliness'),
            'comfort': ('comfort_base', 'score_comfort'),
            'facilities': ('facilities_base', 'score_facilities'),
            'location': ('location_base', 'score_location'),
            'staff': ('staff_base', 'score_staff'),
            'value_for_money': ('value_for_money_base', 'score_value_for_money')
        }
        
        # If aspects specified, use only those; otherwise use all aspects
        if aspects:
            valid_aspects = [a for a in aspects if a in aspect_mapping]
            if not valid_aspects:
                return []
        else:
            valid_aspects = list(aspect_mapping.keys())
        
        # Build aspect fields for both hotels
        aspect_fields = []
        for aspect in valid_aspects:
            base_field, review_field = aspect_mapping[aspect]
            aspect_fields.append(f"h1.{base_field} AS hotel1_{aspect}_base")
            aspect_fields.append(f"h2.{base_field} AS hotel2_{aspect}_base")
            aspect_fields.append(f"avg(r1.{review_field}) AS hotel1_{aspect}_review")
            aspect_fields.append(f"avg(r2.{review_field}) AS hotel2_{aspect}_review")
        
        aspect_select = ",\n               ".join(aspect_fields)
        
        query = f"""
        MATCH (h1:Hotel)-[:LOCATED_IN]->(city1:City)-[:LOCATED_IN]->(country1:Country)
        MATCH (h2:Hotel)-[:LOCATED_IN]->(city2:City)-[:LOCATED_IN]->(country2:Country)
        WHERE toLower(h1.name) = toLower($hotel1) AND toLower(h2.name) = toLower($hotel2)
        OPTIONAL MATCH (h1)<-[:REVIEWED]-(r1:Review)
        OPTIONAL MATCH (h2)<-[:REVIEWED]-(r2:Review)
        RETURN h1.name AS hotel1_name,
               city1.name AS hotel1_city,
               country1.name AS hotel1_country,
               h2.name AS hotel2_name,
               city2.name AS hotel2_city,
               country2.name AS hotel2_country,
               {aspect_select}
        LIMIT 1
        """
        return conn.execute_query(query, {'hotel1': hotel1, 'hotel2': hotel2})
    
    @staticmethod
    def template_C2_compare_with_traveller_type(conn: Neo4jConnection, hotel1: str, hotel2: str, traveller_type: str, aspects: List[str] = None) -> List[Dict[str, Any]]:
        """Template C2: Compare hotels for specific traveller demographic (with optional aspect focus)"""
        aspect_mapping = {
            'cleanliness': ('cleanliness_base', 'score_cleanliness'),
            'comfort': ('comfort_base', 'score_comfort'),
            'facilities': ('facilities_base', 'score_facilities'),
            'location': ('location_base', 'score_location'),
            'staff': ('staff_base', 'score_staff'),
            'value_for_money': ('value_for_money_base', 'score_value_for_money')
        }
        
        # If aspects specified, use only those; otherwise use all aspects
        if aspects:
            valid_aspects = [a for a in aspects if a in aspect_mapping]
            if not valid_aspects:
                # Fallback to C1 with no aspect filter
                return QueryLibrary.template_C1_compare_all_aspects(conn, hotel1, hotel2)
        else:
            valid_aspects = list(aspect_mapping.keys())
        
        # Build aspect fields for both hotels
        aspect_fields = []
        for aspect in valid_aspects:
            base_field, review_field = aspect_mapping[aspect]
            aspect_fields.append(f"h1.{base_field} AS hotel1_{aspect}_base")
            aspect_fields.append(f"h2.{base_field} AS hotel2_{aspect}_base")
            aspect_fields.append(f"avg(r1.{review_field}) AS hotel1_{aspect}_review")
            aspect_fields.append(f"avg(r2.{review_field}) AS hotel2_{aspect}_review")
        
        aspect_select = ",\n               ".join(aspect_fields)
        
        query = f"""
        MATCH (h1:Hotel)-[:LOCATED_IN]->(city1:City)-[:LOCATED_IN]->(country1:Country)
        MATCH (h2:Hotel)-[:LOCATED_IN]->(city2:City)-[:LOCATED_IN]->(country2:Country)
        WHERE toLower(h1.name) = toLower($hotel1) AND toLower(h2.name) = toLower($hotel2)
        OPTIONAL MATCH (h1)<-[:REVIEWED]-(r1:Review)<-[:WROTE]-(t1:Traveller)
        WHERE t1.type = $traveller_type
        OPTIONAL MATCH (h2)<-[:REVIEWED]-(r2:Review)<-[:WROTE]-(t2:Traveller)
        WHERE t2.type = $traveller_type
        WITH h1, h2, city1, city2, country1, country2, collect(r1) AS r1_list, collect(r2) AS r2_list
        WHERE size(r1_list) > 0 AND size(r2_list) > 0
        UNWIND r1_list AS r1
        UNWIND r2_list AS r2
        RETURN h1.name AS hotel1_name,
               city1.name AS hotel1_city,
               country1.name AS hotel1_country,
               h2.name AS hotel2_name,
               city2.name AS hotel2_city,
               country2.name AS hotel2_country,
               {aspect_select}
        LIMIT 1
        """
        results = conn.execute_query(query, {'hotel1': hotel1, 'hotel2': hotel2, 'traveller_type': traveller_type})
        
        # Fallback to template C1 if no results for traveller type
        if not results:
            results = QueryLibrary.template_C1_compare_all_aspects(conn, hotel1, hotel2, aspects)
        
        return results
    
    # ========================================================================
    # VISA CHECK TEMPLATE (1 template)
    # ========================================================================
    
    @staticmethod
    def template_V1_check_visa_requirement(conn: Neo4jConnection, from_country: str, to_country: str) -> List[Dict[str, Any]]:
        """Template V1: Check visa requirement between countries"""
        query = """
        MATCH (from:Country {name: $from_country})
        MATCH (to:Country {name: $to_country})
        OPTIONAL MATCH (from)-[v:NEEDS_VISA]->(to)
        RETURN from.name AS from_country,
               to.name AS to_country,
               v.visa_type AS visa_type,
               CASE WHEN v IS NOT NULL THEN true ELSE false END AS visa_required
        LIMIT 1
        """
        return conn.execute_query(query, {'from_country': from_country, 'to_country': to_country})

print("QueryLibrary class defined with 14 query templates")


QueryLibrary class defined with 14 query templates


In [13]:

# ============================================================================
# QUERY SELECTOR: Route to Appropriate Template Based on Entities
# ============================================================================

def select_and_execute_query(conn: Neo4jConnection, intent: str, entities: Dict[str, Any]) -> List[Dict[str, Any]]:
    """
    Intelligent query selector that chooses the appropriate template based on intent and entities.
    
    Strategy: 
    - Analyze extracted entities to determine which template variant to use
    - Choose most specific template that matches available entities
    - Fall back to simpler templates if specific ones return no results
    
    Returns: Query results from selected template
    """
    
    if intent == "LIST_HOTELS":
        city = entities.get('city')
        country = entities.get('country')
        star_rating = entities.get('star_rating')
        
        # Priority: Most specific first
        if city and star_rating:
            return QueryLibrary.template_L4_list_by_city_and_rating(conn, city, star_rating)  # L4
        elif country and star_rating:
            return QueryLibrary.template_L5_list_by_country_and_rating(conn, country, star_rating)  # L5
        elif city:
            return QueryLibrary.template_L1_list_by_city(conn, city)  # L1
        elif country:
            return QueryLibrary.template_L2_list_by_country(conn, country)  # L2
        elif star_rating:
            return QueryLibrary.template_L3_list_by_rating(conn, star_rating)  # L3
        else:
            return []
    
    elif intent == "RECOMMEND_HOTEL":
        city = entities.get('city')
        traveller_type = entities.get('traveller_type')
        aspects = entities.get('aspects')
        star_rating = entities.get('star_rating')
        age_group = entities.get('age_group')
        user_gender = entities.get('user_gender')
        
        # Priority: Most specific first
        if city:
            if traveller_type and aspects:
                return QueryLibrary.template_R4_recommend_by_traveller_and_aspects(conn, city, traveller_type, aspects, age_group, user_gender)  # R4
            elif aspects:
                return QueryLibrary.template_R3_recommend_by_aspects(conn, city, aspects, age_group, user_gender)  # R3
            elif star_rating:
                return QueryLibrary.template_R5_recommend_with_rating_filter(conn, city, star_rating)  # R5
            else:
                return QueryLibrary.template_R1_recommend_by_location(conn, city)  # R1
        else:
            return []
    
    elif intent == "DESCRIBE_HOTEL":
        hotel_name = entities.get('hotel_name')
        aspects = entities.get('aspects')
        
        if not hotel_name:
            return []
        
        if aspects:
            return QueryLibrary.template_D2_describe_specific_aspects(conn, hotel_name, aspects)  # D2
        else:
            return QueryLibrary.template_D1_describe_all_aspects(conn, hotel_name)  # D1
    
    elif intent == "COMPARE_HOTELS":
        hotel1 = entities.get('hotel1')
        hotel2 = entities.get('hotel2')
        traveller_type = entities.get('traveller_type')
        aspects = entities.get('aspects')
        
        if not hotel1 or not hotel2:
            return []
        
        if traveller_type:
            return QueryLibrary.template_C2_compare_with_traveller_type(conn, hotel1, hotel2, traveller_type, aspects)  # C2
        else:
            return QueryLibrary.template_C1_compare_all_aspects(conn, hotel1, hotel2, aspects)  # C1
    
    elif intent == "CHECK_VISA":
        from_country = entities.get('from_country')
        to_country = entities.get('to_country')
        
        if not from_country or not to_country:
            return []
        
        return QueryLibrary.template_V1_check_visa_requirement(conn, from_country, to_country)  # V1
    
    else:
        return []


### Step 3: Query Router Function

A single entry point that routes to the appropriate query function based on intent.

**Notes on 'aspects' fields:**

- Recommendation templates (R3, R4) compute and return per-aspect *review* averages (e.g., `cleanliness_review`) and a composite score (`composite_aspect_score`). These recommendations use review scores only.
- Description and comparison templates (D2, C1, C2) return both hotel *base* values (e.g., `cleanliness_base`) and aggregated *review* values (e.g., `cleanliness_review`) so you can compare base vs. review-derived performance.

### Template Null/Empty Field Handling

**All templates properly handle missing (None/null) values:**

1. **R3 & R4 (Recommendations)**
   - Accept optional `age_group` and `user_gender` parameters
   - Build WHERE clauses dynamically: only add demographic filters if provided
   - Example: R3 with no demographics executes differently than R3 with `age_group`
   - Fallback: If specific query returns 0 results, falls back to simpler variant

2. **C1 & C2 (Comparisons)**
   - Accept optional `aspects` parameter
   - If `aspects` provided: returns only those aspect comparisons
   - If `aspects` is None: returns all 6 aspects (default behavior)
   - C2 includes intelligent fallback: traveller-type-specific → all aspects

3. **Router Function (select_and_execute_query)**
   - Extracts optional fields from entities dict
   - Passes None values to templates (templates handle them)
   - Example: `age_group = entities.get('age_group')` → None if not extracted

**Parameterized Query Design:**
- All Cypher queries use `$parameter` binding (prevents injection)
- Optional values NOT concatenated into Cypher strings
- Only required parameters in WHERE clauses
- Invalid values filtered by schema enforcement (enforce_schema function)

**Result:**
- All 14 templates work with partial/complete entity sets
- No crashes on missing fields
- Graceful fallbacks when specific queries return 0 results
- 100% functional with real Neo4j data


### Schema-to-Template Alignment

**All schema fields are now used by templates:**

| Intent | Schema Fields | Template Usage | Status |
|--------|---------------|----------------|--------|
| **LIST_HOTELS** | city, country, star_rating | L1-L5: All three used in filters | ✅ Complete |
| **RECOMMEND_HOTEL** | city, country, traveller_type, age_group, user_gender, star_rating, aspects | R1: city; R3/R4: city, traveller_type, age_group, user_gender, aspects; R5: city, star_rating | ✅ Complete |
| **DESCRIBE_HOTEL** | hotel_name, aspects | D1: hotel_name; D2: hotel_name, aspects | ✅ Complete |
| **COMPARE_HOTELS** | hotel1, hotel2, traveller_type, aspects | C1: hotel1, hotel2, aspects; C2: hotel1, hotel2, traveller_type, aspects | ✅ Complete |
| **CHECK_VISA** | from_country, to_country | V1: from_country, to_country | ✅ Complete |

**Field Implementation Details:**

- `age_group` (RECOMMEND_HOTEL): Filters reviews from users of specific age group (R3, R4)
- `user_gender` (RECOMMEND_HOTEL): Filters reviews from users of specific gender (R3, R4)
- `aspects` (RECOMMEND_HOTEL, COMPARE_HOTELS): Focuses recommendations/comparisons on specific aspects (R3, R4, C1, C2)
- All fields are optional (None values handled gracefully)
- All templates include fallback mechanisms for 0-result cases


In [15]:

# ============================================================================
# FINAL TEMPLATE VALIDATION: 14 Working Query Templates
# ============================================================================

print("=" * 140)
print("FINAL QUERY TEMPLATE VALIDATION TEST - 14 Working Templates")
print("=" * 140)

conn = Neo4jConnection()

# Test all 14 working templates
final_tests = [
    ("L1", lambda: QueryLibrary.template_L1_list_by_city(conn, "Paris"), "List by city"),
    ("L2", lambda: QueryLibrary.template_L2_list_by_country(conn, "Egypt"), "List by country"),
    ("L3", lambda: QueryLibrary.template_L3_list_by_rating(conn, 5), "List by rating"),
    ("L4", lambda: QueryLibrary.template_L4_list_by_city_and_rating(conn, "Dubai", 5), "List by city + rating"),
    ("L5", lambda: QueryLibrary.template_L5_list_by_country_and_rating(conn, "Egypt", 5), "List by country + rating"),
    ("R1", lambda: QueryLibrary.template_R1_recommend_by_location(conn, "Cairo"), "Recommend by location"),
    ("R3", lambda: QueryLibrary.template_R3_recommend_by_aspects(conn, "Cairo", ["cleanliness", "location"]), "Recommend by aspects"),
    ("R4", lambda: QueryLibrary.template_R4_recommend_by_traveller_and_aspects(conn, "Istanbul", "family", ["cleanliness"]), "Recommend by traveller + aspects"),
    ("R5", lambda: QueryLibrary.template_R5_recommend_with_rating_filter(conn, "Rome", 4), "Recommend with rating filter"),
    ("D1", lambda: QueryLibrary.template_D1_describe_all_aspects(conn, "The Azure Tower"), "Describe all aspects"),
    ("D2", lambda: QueryLibrary.template_D2_describe_specific_aspects(conn, "Nile Grandeur", ["location", "staff"]), "Describe specific aspects"),
    ("C1", lambda: QueryLibrary.template_C1_compare_all_aspects(conn, "The Azure Tower", "The Royal Compass"), "Compare all aspects"),
    ("C2", lambda: QueryLibrary.template_C2_compare_with_traveller_type(conn, "The Azure Tower", "The Royal Compass", "family"), "Compare with traveller type"),
    ("V1", lambda: QueryLibrary.template_V1_check_visa_requirement(conn, "Egypt", "Turkey"), "Check visa requirement"),
]

print("\nExecuting 14 query templates...")
print("─" * 140)

all_passed = True
for template_id, test_func, description in final_tests:
    try:
        results = test_func()
        status = "✓ PASS" if results else "⊘ EMPTY"
        result_count = len(results)
        if not results:
            all_passed = False
        print(f"{template_id:4s} {status:10s} | {description:40s} → {result_count:2d} result(s)")
    except Exception as e:
        all_passed = False
        print(f"{template_id:4s} ✗ FAIL  | {description:40s} → {str(e)[:40]}...")

print("─" * 140)

conn.close()

if all_passed:
    print("\n✓ ALL 14 TEMPLATES WORKING (no failures)")
else:
    print("\n⚠  Some templates returned no results (but no errors)")

print("\nQUERY LIBRARY FINAL SUMMARY:")
print("=" * 140)
print("""
✓ 14 Parameterized Cypher Query Templates 

FINAL TEMPLATE COUNT:
  LIST_HOTELS:    5 templates (L1, L2, L3, L4, L5)
  RECOMMEND_HOTEL: 4 templates (R1, R3, R4, R5) - R2 removed
  DESCRIBE_HOTEL:  2 templates (D1, D2)
  COMPARE_HOTELS:  2 templates (C1, C2)
  CHECK_VISA:      1 template  (V1)
  ─────────────────────────────────────────
  TOTAL:         14 templates

QUERY SELECTION STRATEGY:
- Router function chooses most specific template based on entities
- Fallback mechanism: R4 tries with traveller type first, then falls back to aspects only
- All queries are parameterized for security
- Templates tested with real data from Neo4j knowledge graph
""")
print("=" * 140)


FINAL QUERY TEMPLATE VALIDATION TEST - 14 Working Templates

Executing 14 query templates...
────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────
L1   ✓ PASS     | List by city                             →  1 result(s)
L2   ✓ PASS     | List by country                          →  1 result(s)
L3   ✓ PASS     | List by rating                           → 25 result(s)
L4   ✓ PASS     | List by city + rating                    →  1 result(s)
L5   ✓ PASS     | List by country + rating                 →  1 result(s)
R1   ✓ PASS     | Recommend by location                    →  1 result(s)
R3   ✓ PASS     | Recommend by aspects                     →  1 result(s)
R4   ✓ PASS     | Recommend by traveller + aspects         →  1 result(s)
R5   ✓ PASS     | Recommend with rating filter             →  1 result(s)
D1   ✓ PASS     | Describe all aspects                     →  1 result(s)
D2   ✓ PASS     | Describe